In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split

In [2]:
with open('locations-length.json') as fopen:
    location = json.load(fopen)

In [3]:
for k, v in location.items():
    v = [i for i in v if 'Nan' not in i]
    location[k] = v

In [4]:
import random

def generate_location(length):
    if length < 3:
        loc = random.sample(location[str(length - 1)], 1)
    else:
        loc = random.sample(location['0'], length)
    return ' '.join(loc)

In [5]:
generate_location(1)

'Sipitang'

In [6]:
# !wget https://raw.githubusercontent.com/nizulzaim/json-malaysia-postalcode/master/postalcode.json

In [7]:
with open('postalcode.json') as fopen:
    postal = json.load(fopen)

In [8]:
len(postal['data'])

54028

In [9]:
def generate_address(row):
    return [row['place'], f"{row['place']} {row['city']}", 
            f"{row['city']} {row['state']}", f"{row['place']} {row['code']}",
           f"{row['place']} {row['code']} {row['city']} {row['state']}"]

In [10]:
addresses = []

for row in postal['data']:
    addresses.extend(generate_address(row))
    
addresses = list(set(addresses))
    
len(addresses)

206029

In [11]:
df = pd.read_csv('senarai-anak-syarikat-kplb.csv')['ALAMAT SYARIKAT'].dropna()
company_address = df.tolist()
company_address = [c.split('TEL')[0].strip() for c in company_address]
company_address = [c.split('Tel')[0].strip().title() for c in company_address]

In [12]:
df = pd.read_csv('penganjur-acara.csv')
for i in range(len(df)):
    row = df.iloc[i]
    l = row['Alamat 1'].replace('\n', ' ').strip().title()
    r = row['Alamat 2'].replace('\n', ' ').strip().title()
    company_address.extend([l, f'{l}, {r}'])

In [13]:
df = pd.read_csv('maklumat-syarikat-jualan-langsung.csv')['ALAMAT TERKINI (TELEFON & FAKS)'].dropna()
company_address.extend(df.tolist())

company_address = list(set(company_address))

In [14]:
company_address = [r.strip().title() for r in company_address]
len(company_address)

1781

In [15]:
import re

comp_address = []
for r in company_address:
    r = r.replace('\n', ' ')
    r = re.sub(r'[ ]+', ' ', r).strip().title()
    for t in ['(T', '( T', '(0', '9T', '(N', 'Tel']:
        if t in r:
            r = r.split(t)
            comp_address.append(r[0].strip())
            break
    else:
        comp_address.append(r)

In [16]:
len(comp_address)

1781

In [17]:
with open('entities-data-v4.json') as fopen:
    entities = json.load(fopen)

In [18]:
results = []
i = 0
while i < len(entities['label']):
    r = []
    if entities['label'][i] == 'location':
        while entities['label'][i] == 'location':
            r.append(i)
            i += 1
        print(r)
        results.append(r)
    i += 1

[145]
[283, 284, 285, 286]
[315]
[336]
[373]
[430, 431, 432]
[476]
[481]
[487]
[492]
[498]
[503]
[511]
[518]
[523]
[531]
[550, 551]
[594]
[689]
[832, 833, 834, 835]
[963, 964, 965, 966]
[968, 969, 970, 971]
[1058, 1059, 1060, 1061, 1062, 1063]
[1100, 1101]
[1246]
[1295, 1296]
[1316, 1317, 1318]
[1465, 1466, 1467]
[1972]
[2095]
[2120, 2121, 2122, 2123]
[2135]
[2151]
[2184]
[2308, 2309, 2310]
[2757, 2758, 2759]
[2847, 2848, 2849, 2850]
[2910, 2911]
[3251]
[3262]
[3280]
[3290]
[3298]
[3308]
[3323]
[3523, 3524, 3525, 3526]
[3627, 3628]
[3761, 3762]
[3765, 3766]
[3800]
[3814]
[3835]
[3852]
[3862]
[4036, 4037]
[4115, 4116]
[4118, 4119]
[4121]
[4124]
[4146]
[4189, 4190, 4191]
[4193]
[4195, 4196]
[4218, 4219]
[4232]
[4325, 4326]
[4364, 4365]
[4367, 4368]
[4370, 4371]
[4374, 4375]
[4378, 4379]
[4409, 4410, 4411, 4412, 4413]
[4415]
[4417, 4418]
[4599, 4600]
[4770, 4771]
[4773]
[4948, 4949]
[4951]
[5071, 5072, 5073, 5074]
[5076, 5077]
[5279, 5280, 5281, 5282]
[5310, 5311, 5312, 5313, 5314, 5315, 

In [19]:
import math

def generate_index(l, name, texts, labels, length):
    cp, indices = [], []
    b = length - len(l)
    left = math.ceil(b / 2)
    right = b - left
    minus = l[0] - left
    if minus < 0:
        absolute = np.abs(minus)
        right += absolute
        left -= absolute

    for i in range(l[0] - left, l[0]):
        cp.append(texts[i])
        indices.append(labels[i])

    cp.extend(name)
    indices.extend([labels[l[0]] for _ in range(len(name))])
    try:
        for i in range(l[-1] + 1, l[-1] + right + 1):
            cp.append(texts[i])
            indices.append(labels[i])
    except Exception as e:
        print(e)
        pass
    
    return cp, indices

In [20]:
train_results, test_results = train_test_split(results, test_size = 0.2)

In [21]:
train_addresses, test_addresses = train_test_split(addresses, test_size = 0.2)

In [22]:
train_comp_address, test_comp_address = train_test_split(comp_address, test_size = 0.2)

In [23]:
import random

In [24]:
train_X, train_Y = [], []

for t in train_addresses:
    x, y = generate_index(train_results[random.randint(0, len(train_results) - 1)], 
                          t.split(), entities['text'], entities['label'], 25)
    if len(x) != len(y):
        print('len not same')
        continue
    train_X.append(x)
    train_Y.append(y)

In [25]:
for t in train_comp_address:
    x, y = generate_index(train_results[random.randint(0, len(train_results) - 1)], 
                          t.split(), entities['text'], entities['label'], 25)
    if len(x) != len(y):
        print('len not same')
        continue
    train_X.append(x)
    train_Y.append(y)

In [26]:
test_X, test_Y = [], []

for t in test_addresses:
    x, y = generate_index(test_results[random.randint(0, len(test_results) - 1)], 
                          t.split(), entities['text'], entities['label'], 25)
    if len(x) != len(y):
        print('len not same')
        continue
    test_X.append(x)
    test_Y.append(y)

In [27]:
for t in test_comp_address:
    x, y = generate_index(test_results[random.randint(0, len(test_results) - 1)], 
                          t.split(), entities['text'], entities['label'], 25)
    if len(x) != len(y):
        print('len not same')
        continue
    test_X.append(x)
    test_Y.append(y)

In [28]:
len(train_X), len(test_X)

(166247, 41563)

In [29]:
with open('location-augmentation.json', 'w') as fopen:
    json.dump({'train_X': train_X, 'train_Y': train_Y, 'test_X': test_X, 'test_Y': test_Y}, fopen)